In [4]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

In [5]:
def extract_text_from_pdf(filename, page_number=None, min_line_length=1):
    """Extract text from a PDF file.
    Args:
        filename (str): Path to the PDF file.
        page_number (int, optional): Page number to extract. If None, extracts all pages.
        min_line_length (int, optional): Minimum length of a line to be included in the output.
    Returns:
        str: Extracted text.
    """
    paragraphs = []
    buffer, full_text = '', ''
    # 提取全部文本
    for i, page_layout in enumerate(extract_pages(filename)):
        if page_number is not None and i != page_number:
            continue
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                full_text += element.get_text() + '\n'
    # 按空行分隔，将文本重新组织成段落
    for line in full_text.split('\n'):
        if len(line) >= min_line_length:
            buffer += (' ' + line) if not line.endswith('-') else line.strip('-')
        elif buffer:
            paragraphs.append(buffer.strip())
            buffer = ''
    if buffer:
        paragraphs.append(buffer.strip())
    return paragraphs            

In [10]:
paragraphs = extract_text_from_pdf("file/LeetCode 101 - A Grinding Guide.pdf", min_line_length=30)
for paragraph in paragraphs[:5]:
    print(paragraph + '\n')


LeetCode 101: A Grinding Guide (Second Edition)

版本：正式版 2.0c，最新版见 GitHub changgyhub/leetcode_101

一个面向有一定的编程基础，但缺乏刷题经验的读者的教科书和工具书。

2019 年底，本书第一版在 GitHub 上正式发布，反响十分热烈。过去的五年，作者积累了大

量的工作经验，同时也越来越觉得这本书存在很多纰漏和不完善之处。于是在 2024 年底，作者



## 定义模型

In [57]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.env_util import *
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

def get_completion(prompt):
    message = [{"role": "user", "content": prompt}]
    model = ChatOpenAI(
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        model_name=os.getenv("MODEL_NAME"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        temperature=0.0,
    )
    return model.invoke(message)

get_completion("你是谁")

AIMessage(content='\n\n你好！我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我能够回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我熟练掌握多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。\n\n如果你有任何问题或需要帮助，随时告诉我！😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 290, 'prompt_tokens': 12, 'total_tokens': 302, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 205, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '01968030c05b25b7cb43503f7be9c1f6', 'finish_reason': None, 'logprobs': None}, id='run-67e9c66e-e3e6-4043-9932-d14203a02193-0', usage_metadata={'input_tokens': 12, 'output_tokens': 290, 'total_tokens': 302, 'input_token_details': {}, 'output_token_details': {'reasoning': 205}})

## Prompt 模板

In [58]:
prompt_template = """
你是一个问答机器人，你的任务是根据下述给定的已知信息回答用户问题。如果你不知道答案，就回答不知道，不要胡编乱造。

已知信息：
{context}

用户问题：
{query}

如果已知信息不包含用户问题的答案，或者已知信息不足以回答用户的问题，就回答不知道，不要胡编乱造。
请不要输出已知信息中不包含的信息或答案。
请用中文回答用户问题。
"""

def build_prompt(prompt_template, **kwargs):
    """
    自定义参数渲染 Prompt 模板
    """
    inputs = {}
    for k, v in kwargs.items():
        if isinstance(v, list) and all(isinstance(elem, str) for elem in v):
            val = "\n\n".join(v)
        else:
            val = v
        inputs[k] = val
    return prompt_template.format(**inputs)


## 向量间相似度的计算

余弦相似度通过向量点积与模长乘积的比值计算，公式如下： 

$$
\cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|} = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \sqrt{\sum_{i=1}^{n} B_i^2}}
$$

L2 距离是向量对应元素差的平方和的平方根，公式如下：

$$
d(\mathbf{A}, \mathbf{B}) = \sqrt{\sum_{i=1}^{n} (A_i - B_i)^2}
$$

In [59]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(A, B):
    ''' 余弦相似度（越大越相似） '''
    return dot(A, B)/(norm(A)*norm(B))

def l2_distance(A, B):
    ''' 欧式距离（越小越相似） '''
    x = np.asarray(A)
    y = np.asarray(B)
    return norm(x-y)

## 嵌入模型

[嵌入模型库](https://www.modelscope.cn/models?page=1&tasks=sentence-embedding&type=nlp)

标准

- **找需求相关的语料库来进行文本向量转换测试，进行评估**
- 大多数场景下，开源的嵌入模型使用效果都一般，要进行检索召回率，建议对模型进行微调
- 嵌入模型的维度越高，表示特征细节提取越丰富

In [62]:
from typing import List
from langchain_openai import OpenAIEmbeddings
from volcenginesdkarkruntime import Ark

def get_embeddings(texts: List[str]):
    ''' OpenAI Embeddings '''
    embedding_model = OpenAIEmbeddings(
        api_key=os.getenv('OPENAI_API_KEY'),
        base_url=os.getenv('OPENAI_BASE_URL'),
        model=os.getenv('EMBEDDING_MODEL_NAME')
    )
    return embedding_model.embed_documents(texts)

def get_ark_embeddings(texts: List[str]):
    client = Ark(api_key=os.getenv('ARK_API_KEY'))
    return client.embeddings.create(
        model="doubao-embedding-large-text-240915",
        input=texts
    )

test_texts = ['吃完海鲜可以喝牛奶吗？']
vec = get_embeddings(test_texts)[0]
print(f'Total Dimension: {len(vec)}')
print(f'Fist 10 Dimensions: {vec[:10]}')

vec = get_ark_embeddings(test_texts).data[0].embedding
print(f'\n[ARK] Total Dimension: {len(vec)}')
print(f'[ARK] Fist 10 Dimensions: {vec[:10]}')

Total Dimension: 1024
Fist 10 Dimensions: [0.05262557417154312, 0.02764252945780754, 0.07897865772247314, -0.0029994763899594545, 0.0073790643364191055, 0.026514261960983276, 0.009670855477452278, -0.0016747706104069948, 0.045372430235147476, 0.004772466607391834]

[ARK] Total Dimension: 4096
[ARK] Fist 10 Dimensions: [0.2421875, 3.859375, -0.482421875, -3.96875, 0.74609375, -2.625, 1.6015625, 3.359375, 0.3984375, 3.25]


In [63]:
query = "国际争端"
# 前两条为国际争端
documents = [
    "联合国就苏丹达尔富地区大规模暴力事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典“入约”问题进行谈判",
    "日本歧阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营",
    "我国首次在空间站开展舱外辐射生物学暴露实验"
]

# query_vector = get_embeddings([query])[0]
# doc_vectors = get_embeddings(documents)

# 实测 doubao Embedding 模型效果更好
query_vector = get_ark_embeddings([query]).data[0].embedding
doc_vectors = [doc_vector.embedding for doc_vector in get_ark_embeddings(documents).data]

# 余弦距离越大表示越相似
print(f'query 与自己的余弦距离为：{cosine_similarity(query_vector, query_vector)}')
print('query 与文档的余弦距离为：')
for doc_vector in doc_vectors:
    print(f'{cosine_similarity(query_vector, doc_vector)}')

print('*' * 80)

# L2距离（欧式距离）越小表示越相似
print(f'query 与自己的L2距离为：{l2_distance(query_vector, query_vector)}')
print('query 与文档的L2距离为：')
for doc_vector in doc_vectors:
    print(f'{l2_distance(query_vector, doc_vector)}')

query 与自己的余弦距离为：1.0
query 与文档的余弦距离为：
0.8851309597447
0.8601865743943213
0.802517564858825
0.7592321142985579
0.796052817036916
********************************************************************************
query 与自己的L2距离为：0.0
query 与文档的L2距离为：
63.85760211053751
69.9480541076191
82.65289032621875
91.29779700506289
84.24111387780202
